In [1]:
# !pip install torch

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss

from tqdm.auto import tqdm
import copy

np.random.seed(12345)
torch.manual_seed(123)

# ========== Data ==========

In [3]:
data = pd.read_csv('dataset3.csv')

In [4]:
data

x1  x2  y
0    0   4  A
1    2   2  B
2    2   4  A
3    1   4  A
4   -1   3  A
5    0   2  A
6    3   1  B
7   -3   1  C
8    3   3  B
9    1  -2  C
10   1  -1  C
11   0  -1  C
12   3   2  B
13  -2   5  A
14   4   3  B
15   1   1  B
16   1   3  A
17  -1  -2  C
18   3   0  B
19  -1  -1  C
20  -2  -2  C
21  -1  -3  C
22  -1   4  A
23   0  -2  C
24   0   3  A
25   3  -1  B
26   4  -1  B

In [5]:
X = data[['x1', 'x2']].to_numpy()

In [6]:
y = data['y'].to_numpy()

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True)

In [8]:
le = LabelEncoder()
y_le_train = le.fit_transform(y_train)
y_le_val = le.transform(y_train)

# ========== Model ==========

In [9]:
if torch.cuda.is_available():
  device = 'cuda:0'
else:
  device = 'cpu'

In [10]:
X_train = torch.from_numpy(X_train).float().to(device)
y_le_train = torch.from_numpy(y_le_train).long().to(device)

X_val = torch.from_numpy(X_val).float().to(device)
y_le_val = torch.from_numpy(y_le_val).long().to(device)

In [11]:
layers = []

layers.append(nn.Linear(X.shape[1], 5))
layers.append(nn.ReLU())

layers.append(nn.Linear(5, 4))
layers.append(nn.ReLU())

layers.append(nn.Linear(4, 3))

model = nn.Sequential(*layers).to(device)

In [12]:
best_loss = np.inf

l1_ratios = [0.01, 0.5, 0.75, 1]
alphas = np.logspace(-3, 2, 100)

for l1_ratio in tqdm(l1_ratios):
    for alpha in tqdm(alphas):
        optimizer = torch.optim.SGD(model.parameters(), lr=1)
        loss_function = nn.CrossEntropyLoss()
        for i in tqdm(range(500)):
            model.train()
            optimizer.zero_grad()

            y_pred_softmax_train = model(X_train)

            l1_norm = 0
            l2_norm = 0
            for layer in model.children():
                if isinstance(layer, nn.Linear):
                    for params in layer.parameters():
                        l1_norm += sum(p.abs().sum() for p in params)
                        l2_norm += sum(p.pow(2.0).sum() for p in params)

            l1_norm = alpha*l1_ratio*l1_norm
            l2_norm = alpha*(1-l1_ratio)*l2_norm
            loss = loss_function(y_pred_softmax_train, y_le_train) + l1_norm + l2_norm
            loss.backward()

            optimizer.step()

        with torch.no_grad():
            model.eval()
            y_pred_softmax_val = torch.log_softmax(model(X_val), dim=1)

        if np.isnan(y_pred_softmax_val.min()):
            continue

        loss = log_loss(y_val, y_pred_softmax_val, labels=np.unique(y))

        if loss < best_loss:
            best_loss = loss
            best_model = copy.deepcopy(model)
            best_l1_ratio = l1_ratio
            best_alpha = alpha

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [13]:
best_l1_ratio

0.01

In [14]:
best_alpha

0.001

In [15]:
for name, param in best_model.named_parameters():
    print(name, param)
    print()

0.weight Parameter containing:
tensor([[-0.4405, -0.0392],
        [-0.6480,  0.6184],
        [-0.7204,  0.8042],
        [-1.0414, -1.1448],
        [-0.6497,  0.7967]], requires_grad=True)

0.bias Parameter containing:
tensor([ 0.0781,  0.0634, -0.1567,  1.1358, -0.1293], requires_grad=True)

2.weight Parameter containing:
tensor([[-0.0312, -0.1447,  0.0786, -0.0360,  0.0295],
        [ 0.3695,  0.0895,  0.0719,  1.5833, -0.0878],
        [ 0.2800, -0.0247,  0.1273,  1.0360, -0.1777],
        [-0.0128,  0.8767,  1.0311, -0.2058,  1.0072]], requires_grad=True)

2.bias Parameter containing:
tensor([-0.0320, -0.0012,  0.1218, -0.1715], requires_grad=True)

4.weight Parameter containing:
tensor([[ 0.0180, -0.4773, -0.7357,  1.2735],
        [ 0.1689, -0.7529, -0.0330, -1.1046],
        [ 0.0209,  1.3495,  0.7884, -0.1457]], requires_grad=True)

4.bias Parameter containing:
tensor([-1.2911,  2.9887, -1.5387], requires_grad=True)



# ========== Prediction ==========

In [16]:
X = pd.read_csv('incoming_data3.csv').to_numpy()

In [17]:
X = torch.from_numpy(X).float().to(device)

In [18]:
with torch.no_grad():
    best_model.eval()
    y_pred_softmax = torch.log_softmax(best_model(X), dim=1)
    _, y_pred_le = torch.max(y_pred_softmax, dim=1)
y_pred = le.inverse_transform(y_pred_le)

y_pred

array(['C', 'A', 'B', 'A'], dtype=object)